In [1]:
:set -XRankNTypes
:set -XTypeOperators
:set -XKindSignatures
:set -XInstanceSigs
:set -XLambdaCase

In [2]:
f :: a -> b
f = undefined

In [3]:
type P = (->)

In [4]:


f' :: a `P` b
f' = undefined

f'' :: P a b
f'' = undefined

generateA :: i -> a
generateA = undefined

useB :: b -> o
useB = undefined

f''' :: (i -> a) -> (b -> o) -> P a b -> P i o
f''' g u f = u . f . g

class Profunctor (p :: * -> * -> *) where
  dimap :: (i -> a) -> (b -> o) -> p a b -> p i o 
  
instance Profunctor (->) where
    dimap g u f = u . f . g



So how to get a `Functor` be a `Profunctor`. For that we need a new data type that somehow talks about `Functor f`, the type contained within `f` i.e `a`, and the other type which is required for `Profunctor` i.e. `b`. Let's call it `Star` (we will get to the reason later)

In [5]:

-- if you look closely, a here is in covariant position and b in contra
newtype Star f a b = Star {runStar :: a -> f b} 

-- now lets lift the Functor 
instance Functor f => Profunctor (Star f) where
    dimap :: (i -> a) -> (b -> o) -> Star f a b -> Star f i o
    dimap f g s = Star (fmap g . runStar s . f)

So `Star` is another `Profunctor` like `(->)`. Let's just see one more. Like `Star` which uses all three type variables `f`, `a`, and `b`, this one will take 3 but use only 2 in a way that will enable us to make a `Profunctor` out of it. Since it forgets a type in its implementation, we will call it `Forget`

In [6]:
newtype Forget f a b = Forget {runForget :: a -> f} -- again, a here is covariant, f is contravariant, and b is forgotten

-- hence for it to be a Profunctor we don't need the Functor constraint like Star
instance Profunctor (Forget f) where
    dimap :: (i -> a) -> (b -> o) -> Forget f a b -> Forget f i o
    dimap f _ fo = Forget (runForget fo . f)


As `functors` provide ways to map objects of a container (read a data structure), `natural transformations` provide ways to map `functors` i.e `type Natural f g = forall a. f a -> g a`. Of course, `natural transformations` need to adhere to some laws. For `profunctors`, `natural transformations` happen in couple of ways (one loose way to reason it is as profunctors deal with 2 type parameters).

**First**, is when both type parameters are lifted to a `product` type i.e. `p a b |-> p (a, c) (b, c)` or `p a b |-> p (c, a) (c, b)`. Here, the type parameter `c` can be considered to be providing additional information as a result of the transformation. It's encoded in a type class named `Strong`.

In [7]:
class Profunctor p => Strong p where
    first' :: p a b -> p (a, c) (b, c)
    second' :: p a b -> p (c, a) (c, b)
    
-- let's lift (->) to be a Strong one
instance Strong (->) where
    first' f = \(a, c) -> (f a, c)
    second' f = \(c, a) -> (c, f a)
    
-- let's lift Star to be a Strong one
instance Functor f => Strong (Star f) where
    first' :: Star f a b -> Star f (a, c) (b, c)
    first' s = Star (\(a, c) -> fmap (\b -> (b, c)) $ runStar s a)
    
    second' :: Star f a b -> Star f (c, a) (c, b)
    second' s = Star (\(c, a) -> fmap (\b -> (c, b)) $ runStar s a)
    
-- simillarly, let's lift Forget to be a Strong one
instance Strong (Forget f) where
    first' fo = Forget (runForget fo . fst)
    second' fo = Forget (runForget fo . snd)

__Second__, when both type parameters are lifted to a `sum` type i.e. `p a b -> p (Either a c) (Either b c)` or `p a b -> p (Either c a) (Either c b)`. Here the type parameter `c` can be considered to be `maybe` providing additional information. It's encoded in a type class named `Choice`

In [8]:

class Profunctor p => Choice p where
    left' :: p a b -> p (Either a c) (Either b c)
    right' :: p a b -> p (Either c a) (Either c b)
    
instance Choice (->) where
    left' f (Left a) = Left (f a)
    left' _ (Right c) = Right c
    
    right' = fmap
    
-- we need Applicative constraint to lift Right
instance Applicative f => Choice (Star f) where
    left' (Star s) = Star $ either (fmap Left . s) (pure . Right)
    
    right' (Star s) = Star $ either (pure . Left) (fmap Right . s)
    
-- we need Monoid constraint to get mempty for the forgotten case
instance Monoid fo => Choice (Forget fo) where
    left' (Forget fo) = Forget $ either fo (const mempty)
    
    right' (Forget fo) = Forget $ either (const mempty) fo

`Strong` and `Choice` above are also seen as refinements of `Profunctor` which concerns its interaction with sum and product types.

In [9]:
cross :: (a -> c) -> (b -> d) -> (a , b) -> (c, d)
cross f g (a, c) = (f a, g c)

class Profunctor p => Monoidal p where
    par :: p a b -> p c d -> p (a,c) (b,d)
    empty :: p () ()
    
instance Monoidal (->) where
    empty = id
    par = cross
    
instance Applicative f => Monoidal (Star f) where
    empty = Star pure
    par (Star f) (Star g) = Star (\(a,c) -> (,) <$> f a <*> g c)
    
instance Monoid f => Monoidal (Forget f) where
    empty = Forget (const mempty)
    par (Forget f) (Forget g) = Forget (\(a,c) -> f a <> g c)

#### Profunctor Optics

In [10]:
type Optic p a b s t = p a b -> p s t

Profunctor Optics : Modular Data Accessors. Matthew Pickering, Jeremy Gibbons, and Nicolas Wua

> When `S` is a composite type with some component of type `A`, and `T` similarly a composite type in which that component has type `B`, and `P` is some notion of `transformer`, then we can think of a data accessor of type `Optic P A B S T` as lifting a component transformer of type `P A B` to a whole-structure transformer of type `P S T`. We will retrieve equivalents of our original definitions of lens, prism, and so on by placing various constraints on `P`

**Adapters**

Adapters are the concrete representation of optics in which the things which is viewed or matched is the whole structure. hence it just need to pair of functions i.e. one for view/match `S -> A` and one for update/build `B -> T`

In [11]:


data Adapter a b s t = Adapter {from :: s -> a, to :: b -> t}
-- profunctor constraint to note that polymorphically the notion of an abstracter is simply a function from `P A B` to `P S T`
type AdapterP a b s t = forall p. Profunctor p => Optic p a b s t

Establishing isomorphism between `Adapter` and `AdapterP`

In [12]:
-- the type is equivalent to
-- Profunctor p => Adapter a b s t -> p a b -> p s t
adapterC2P :: Adapter a b s t -> AdapterP a b s t
adapterC2P (Adapter f t) = dimap f t

instance Profunctor (Adapter a b) where
    dimap :: (i -> c) -> (d -> o) -> Adapter a b c d -> Adapter a b i o
    dimap f g (Adapter r s) = Adapter (r . f) (g . s)

-- the type is equivalent to
-- Profunctor p q => p a b -> p s t -> Adapter a b s t
adapterP2C :: AdapterP a b s t -> Adapter a b s t
adapterP2C f = f (Adapter id id)

**Lens**

Lenses provide ways to

- _view_ the contained type `A` inside  the container type `S` and 
- _update_ the container type `S` to a container type `T` having elements of contained type `B`


In [13]:
data Lens a b s t = Lens {view :: s -> a, update :: (b, s) -> t}

-- Strong profunctors produce the notion of lenses as there is product type involved here
type LensP a b s t = forall p. Strong p => Optic p a b s t

Establishing isomorphism between `Lens` and `LensP`

In [14]:
lensC2P :: Lens a b s t -> LensP a b s t
lensC2P (Lens v u) = dimap (\s -> (v s, s)) u . first'

instance Profunctor (Lens a b) where
    dimap :: (i -> c) -> (d -> o) -> Lens a b c d -> Lens a b i o
    dimap f g (Lens v u) = Lens (v . f) (\(b,i) -> g $ u (b, f i))

instance Strong (Lens a b) where
    first' :: Lens a b c d -> Lens a b (c, e) (d, e)
    first' (Lens v u) = Lens (v . fst) (\(b, (c, e)) -> (u (b, c), e))
    
    second' :: Lens a b c d -> Lens a b (e, c) (e, d)
    second' (Lens v u) = Lens (v . snd) (\(b, (e, c)) -> (e, u (b, c)))

lensP2C :: LensP a b s t -> Lens a b s t
lensP2C f = f (Lens id fst)

**Prisms**

Prisms provide ways to
  - _match_ the probable contained type `A` (and may be transform it to `B`) inside the compound type `S` (if no match then it reverts to `S` or may be transform to `T`) and 
  - _build_ the compound type `T` from one of the contained types `B`


In [15]:
data Prism a b s t = Prism {match :: s -> Either t a, build :: b -> t}

-- Choice profunctors produce the notion of prisms as there is sum type involved here
type PrismP a b s t = forall p. Choice p => Optic p a b s t

Establishing isomorphism between `Prism` and `PrismP`

In [16]:
prismC2P :: Prism a b s t -> PrismP a b s t
prismC2P (Prism m b) = dimap m (either id b) . right'

instance Profunctor (Prism a b) where
    dimap :: (i -> c) -> (d -> o) -> Prism a b c d -> Prism a b i o
    dimap f g (Prism m b) = Prism (either (Left . g) (Right) . m . f) (g . b)
    
instance Choice (Prism a b) where
    left' :: Prism a b c d -> Prism a b (Either c e) (Either d e)
    left' (Prism m b) = Prism (either (either (Left . Left) Right . m) (Left . Right))
                              (Left . b)
    
    right' :: Prism a b c d -> Prism a b (Either e c) (Either e d)
    right' (Prism m b) = Prism (either (Left . Left) (either (Left . Right) Right . m))
                               (Right . b)

prismP2C :: PrismP a b s t -> Prism a b s t
prismP2C f = f (Prism Right id)

**Traversals**

- traversal datatype is a container with finite number of elements having an ordering. It can be _traversed_ whereby each element is visited in the given order

- a container type `S` with elements of type `A` is a _traversal_ when there exists types `B`, `T`, and a traversal function of type `(A -> F B) -> (S -> F T)` for each applicative functor `F` (additionally satisfying some traversal laws)

- as a **generalisation of lenses and prisms**, traversals provides access not just to a single component within a whole structure but onto an entire sequence of such components

- traversal type `S` is equivalent to `∃n. A^n x (B^n -> T)`, where `n` is the number of elements in `S`. This tupling represents both yielding of sequence of elements inside container in the order mentioned and also replacing the container with a sequence of new elements

- above equivalence can be captured as a datatype

In [17]:
data FunList a b t = Done t | More a (FunList a b (b -> t))

instance Functor (FunList a b) where
    fmap f (Done t) = Done $ f t
    fmap f (More x l) = More x (fmap (f .) l)
    
instance Applicative (FunList a b) where
    pure = Done
    Done f <*> f' = fmap f f'
    More x l <*> f' = More x (flip <$> l <*> f')

This datatype also is isomorphic to `Either T (A , FunList A B (B -> T))`

In [18]:
out :: FunList a b t -> Either t (a, FunList a b (b -> t))
out (Done t) = Left t
out (More a l) = Right (a, l)

inn :: Either t (a, FunList a b (b -> t)) -> FunList a b t
inn = either Done (\(a,l) -> More a l)

Based on the above isomorphism, it can be proved that traversal function of type `(A -> F B) -> (S -> F T)` for each applicative functor `F` yields an isomorphism `S ≍ FunList A B T`

The definition of concrete traversal then is

In [22]:
newtype Traversal a b s t = Traversal {extract::s -> FunList a b t}

-- traversal being the generalisation, we need all notions of sum, product, and empty
type TraversalP a b s t = forall p. (Strong p, Choice p, Monoidal p) => Optic p a b s t

To establish the isomorphism we need a notion of `traverse` that lifts a transformation of `A`s to `B`s to act on each of the elements of `FunList`

In [27]:
traverse :: (Choice p, Monoidal p) => p a b -> p (FunList a c t) (FunList b c t)
traverse k = dimap out inn (right' $ par k $ traverse k)

In [52]:
fuse :: FunList b b t -> t
fuse (Done t) = t
fuse (More x l) = fuse l x

single :: a -> FunList a b b
single x = More x (Done id)

traversalC2P :: Traversal a b s t -> TraversalP a b s t
traversalC2P (Traversal f) p = dimap f fuse (traverse p)

instance Profunctor (Traversal a b) where
    dimap :: (i -> c) -> (d -> o) -> Traversal a b c d -> Traversal a b i o
    dimap f g (Traversal h) = Traversal (fmap g . h . f)

instance Strong (Traversal a b) where
    first' :: Traversal a b c d -> Traversal a b (c, e) (d, e)
    first' (Traversal s) = Traversal (\(c,e) -> (,) <$> s c <*> pure e)
    
    second' :: Traversal a b c d -> Traversal a b (e, c) (e, d)
    second' (Traversal s) = Traversal (\(e, c) -> (,) <$> pure e <*> s c) 
    
instance Choice (Traversal a b) where
    left' :: Traversal a b c d -> Traversal a b (Either c e) (Either d e)
    left' (Traversal s) = Traversal (either (fmap Left . s) (fmap Right . pure))
    
    right' :: Traversal a b c d -> Traversal a b (Either e c) (Either e d)
    right' (Traversal s) = Traversal (either (fmap Left . pure) (fmap Right . s))
    
instance Monoidal (Traversal a b) where
    empty :: Traversal a b () ()
    empty = Traversal pure
    
    par :: Traversal a b c d -> Traversal a b e f -> Traversal a b (c, e) (d, f)
    par (Traversal s) (Traversal t) = Traversal (\(c,e) -> (,) <$> s c <*> t e)

traversalP2C :: TraversalP a b s t -> Traversal a b s t
traversalP2C f = f $ Traversal single

In [54]:
traverseOf :: TraversalP a b s t -> (forall f. Applicative f => (a -> f b) -> (s -> f t))
traverseOf p f = runStar $ p $ Star f 